## importing_libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

## importing_data

In [2]:
csv_file = "C://Users//sdkay//Documents//INPT//2A//s4//PFA//prj_tech//data//fer2013.csv"

df = pd.read_csv(csv_file)

In [3]:
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [4]:
df.dtypes

emotion     int64
pixels     object
Usage      object
dtype: object

In [5]:
df.Usage.unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

## raw_train_set

In [6]:
df_train = df[df["Usage"] == "Training"]

In [7]:
raw_X_train = df_train["pixels"].to_list()
raw_y_train = df_train["emotion"].to_list()

In [8]:
len(raw_X_train)

28709

In [9]:
len(raw_X_train[0].split())**0.5

48.0

In [10]:
len(raw_y_train)

28709

## raw_test_set

In [11]:
df_test = df[df["Usage"] != "Training"]

In [12]:
raw_X_test = df_test["pixels"].to_list()
raw_y_test = df_test["emotion"].to_list()

In [13]:
len(raw_X_test)

7178

In [14]:
len(raw_y_test)

7178

## train_set

In [15]:
X_train = []
y_train = np.array(raw_y_train)

for e in raw_X_train : 
    X_train.append(list(map(int, e.split())))

X_train = np.array(X_train).reshape((28709 , 48 , 48 , 1))

## test_set

In [16]:
X_test = []
y_test = np.array(raw_y_test)

for e in raw_X_test : 
        X_test.append(list(map(int, e.split())))

X_test = np.array(X_test).reshape((7178 , 48 , 48 , 1))
    

In [17]:
classes = ['Angry' , 'Disgust' , 'Fear' , 'Happy' , 'Sad' , 'Surprise' , 'Neutral'] 

## normalisation

In [18]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [23]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(
                                                                                                            filters = 16, kernel_size = (3,3) , activation = 'relu' , input_shape = (48,48,1)),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Conv2D(filters = 32,kernel_size = (3,3) , activation = 'relu' , input_shape = (48,48,1)),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3) , activation = 'relu'),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3) , activation = 'relu'),
                                                                                                            tf.keras.layers.MaxPool2D(2,2),

                                                                                                            tf.keras.layers.Flatten(),

                                                                                                            tf.keras.layers.Dense(100 , activation = 'relu'),
                                                                                                            tf.keras.layers.Dense(7 , activation = 'softmax')                          
                                                                                                            ])

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 46, 46, 16)        160       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 21, 21, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 128)        

In [25]:
model.compile(
                          optimizer = tf.keras.optimizers.RMSprop(learning_rate = .00001),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=10 , validation_split = .1 )

Epoch 1/10


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_2/conv2d_8/Conv2D (defined at <ipython-input-26-c2badafb7290>:1) ]] [Op:__inference_train_function_1404]

Function call stack:
train_function


In [43]:
string_list = ["1", "2", "3"]

integer_map = map(int, string_list)
integer_list = list(integer_map)
print(integer_list)

[1, 2, 3]


In [31]:
import pandas as pd

df = pd.DataFrame({'pixels': ['10 20 30 40', '11 12 13 14']})

a = (
    df['pixels'].str.split(' ', expand=True)
        .astype(int).to_numpy()
        .reshape((2, 2, 2, 1))
)
print(a)

[[[[10]
   [20]]

  [[30]
   [40]]]


 [[[11]
   [12]]

  [[13]
   [14]]]]


In [45]:
df["pixels"].str.split(expand = True).astype(int).to_numpy()

array([[10, 20, 30, 40],
       [11, 12, 13, 14]])

In [33]:
a.shape

(2, 2, 2, 1)

In [51]:
df_train['pixels'].str.split(expand = True).astype(int).to_numpy().reshape((28709 , 48 , 48))

array([[[ 70,  80,  82, ...,  52,  43,  41],
        [ 65,  61,  58, ...,  56,  52,  44],
        [ 50,  43,  54, ...,  49,  56,  47],
        ...,
        [ 91,  65,  42, ...,  72,  56,  43],
        [ 77,  82,  79, ..., 105,  70,  46],
        [ 77,  72,  84, ..., 106, 109,  82]],

       [[151, 150, 147, ..., 129, 140, 120],
        [151, 149, 149, ..., 122, 141, 137],
        [151, 151, 156, ..., 109, 123, 146],
        ...,
        [188, 188, 121, ..., 185, 185, 186],
        [188, 187, 196, ..., 186, 182, 187],
        [186, 184, 185, ..., 193, 183, 184]],

       [[231, 212, 156, ...,  44,  27,  16],
        [229, 175, 148, ...,  27,  35,  27],
        [214, 156, 157, ...,  28,  22,  28],
        ...,
        [241, 245, 250, ...,  57, 101, 146],
        [246, 250, 252, ...,  78, 105, 162],
        [250, 251, 250, ...,  88, 110, 152]],

       ...,

       [[ 74,  81,  87, ..., 189, 191, 192],
        [ 78,  82,  89, ..., 185, 189, 193],
        [ 81,  86,  94, ..., 176, 185, 193